In [1]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
dataset=pd.read_excel("C:/Users/10253/Desktop/ML-CW-2/Data/trainDataset.xls")
dataset=dataset[dataset["RelapseFreeSurvival (outcome)"]!=999]
dataset.head(len(dataset))

,ID,pCR (outcome),RelapseFreeSurvival (outcome),Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,TRG002174,1,144.000000,41.0,0,0,0,1,3,3,...,0.517172,0.375126,3.325332,0.002314,3.880772e+06,473.464852,0.000768,0.182615,0.030508,0.000758
1,TRG002178,0,142.000000,39.0,1,1,0,0,3,3,...,0.444391,0.444391,3.032144,0.005612,2.372010e+06,59.459710,0.004383,0.032012,0.001006,0.003685
2,TRG002204,1,135.000000,31.0,0,0,0,1,2,1,...,0.534549,0.534549,2.485848,0.006752,1.540027e+06,33.935384,0.007584,0.024062,0.000529,0.006447
3,TRG002206,0,12.000000,35.0,0,0,0,1,3,3,...,0.506185,0.506185,2.606255,0.003755,6.936741e+06,46.859265,0.005424,0.013707,0.000178,0.004543
4,TRG002210,0,109.000000,61.0,1,0,0,0,2,1,...,0.462282,0.462282,2.809279,0.006521,1.265399e+06,39.621023,0.006585,0.034148,0.001083,0.005626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,TRG002955,1,49.250000,46.1,0,0,0,1,3,3,...,0.439568,0.439568,3.056046,0.001339,1.671271e+07,79.989003,0.003282,0.024716,0.000812,0.003078
396,TRG002958,0,48.500000,53.3,0,0,0,1,2,1,...,0.527779,0.527778,1.500000,0.003728,2.132007e+05,0.996746,0.252582,0.007380,0.000037,0.231059
397,TRG002961,0,47.500000,68.8,1,0,0,0,3,3,...,0.313693,0.313693,3.573557,0.001112,2.008034e+07,204.864200,0.001372,0.054063,0.003697,0.001368
398,TRG002962,0,46.916667,46.0,1,0,0,0,2,1,...,0.670229,0.670229,1.857045,0.006706,5.609262e+05,9.609163,0.026591,0.018682,0.000311,0.022676


In [2]:
dataset.drop_duplicates(inplace=True)

In [3]:
y=dataset['RelapseFreeSurvival (outcome)']
X=dataset.drop(['pCR (outcome)','RelapseFreeSurvival (outcome)'],axis=1)


X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [4]:
class columnDropperTransformer(): #https://stackoverflow.com/questions/68402691/adding-dropping-column-instance-into-a-pipeline
    def __init__(self,columns):
        self.columns=columns

    def fit(self,X,y=None):
        return self

    def transform(self,X,y=None):
        return X.drop(self.columns,axis=1)

    def fit_transform(self,X,y=None):
        return X.drop(self.columns,axis=1)

dropper = columnDropperTransformer(['ID'])
X_train=dropper.fit_transform(X_train)

X_test=dropper.transform(X_test)

In [5]:
dataset.isnull().sum()

ID                               0
pCR (outcome)                    0
RelapseFreeSurvival (outcome)    0
Age                              0
ER                               0
                                ..
original_ngtdm_Busyness          0
original_ngtdm_Coarseness        0
original_ngtdm_Complexity        0
original_ngtdm_Contrast          0
original_ngtdm_Strength          0
Length: 120, dtype: int64

In [6]:
from sklearn.impute import KNNImputer

names=X_train.columns

imp=KNNImputer(missing_values=np.nan,n_neighbors=200)
X_train=imp.fit_transform(X_train)
X_test=imp.transform(X_test)

imp.missing_values=np.float64(999)
X_train=imp.fit_transform(X_train)
X_test=imp.transform(X_test)

In [7]:
class outlierHandlingWithDev():
    def fit(self,X,y=None):
        self.mean=np.zeros(len(X[0]))
        self.std=np.zeros(len(X[0]))
        for i in range(len(X[0])):
            self.mean[i]=X[:,i].mean()
            self.std[i]=X[:,i].std()

        return self

    def transform(self,X,y=None):
        for i in range(len(X)):
            for j in range(len(X[i])):
                if np.abs(X[i][j]-self.mean[j])>2*self.std[j]:
                    X[i][j]=999
        return X

    def fit_transform(self,X,y=None):
        self.fit(X,y)
        return self.transform(X,y)

# detector=outlierHandlingWithDev()
# detector.fit(X_train)
# detector.transform(X_train)
# detector.transform(X_test)
#
# X_train=imp.fit_transform(X_train)
# X_test=imp.transform(X_test)

X_train=pd.DataFrame(X_train,columns=names)
X_test=pd.DataFrame(X_test,columns=names)

In [8]:
# from sklearn.preprocessing import KBinsDiscretizer
# datasetCopy=dataset.iloc[:,3].values.reshape(-1,1)
# # 不能导入一维，所以reshape
# est = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')
# est.fit(datasetCopy)
# Xt = est.transform(datasetCopy)
# set(Xt.ravel())
# dataset.iloc[:,3]=Xt
# dataset

In [9]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train[:]=scaler.fit_transform(X_train)
X_test[:]=scaler.transform(X_test)

In [10]:
corr=X_train.corr()
corr=corr.where(abs(corr)>0.9)
count = 0
corr_fea = []

for i in corr.columns:
    for j in corr.columns:
        if abs(corr[i][j])>0.9 and i!=j:
            if i not in corr_fea:
                count+=1
                corr_fea.append(i)

            if j not in corr_fea:
                count+=1
                corr_fea.append(j)

count

82

In [11]:
def is_related(pair1, pair2):
    if (pair1[0] in pair2 or pair1[1] in pair2) and pair1!=pair2 and not (pair1[0]==pair2[1] and pair1[1]==pair2[0]):
        return True
    return False

def add_relation(corr_fea):
    for i in range(len(corr_fea)):
        for j in range(i,len(corr_fea)):
            if i!=j:
                if is_related(corr_fea[i],corr_fea[j]):
                    fea1=tuple(set(corr_fea[i])-set(corr_fea[j]))
                    fea2=tuple(set(corr_fea[j])-set(corr_fea[i]))
                    fea=fea1+fea2
                    if fea not in corr_fea and rev(fea) not in corr_fea:
                        corr_fea.append((fea1+fea2))

def rev(fea):
    f1=fea[0]
    f2=fea[1]

    return (f2,f1)

corr=X_train.corr()
corr=corr.where(abs(corr)>0.9)
count=0
corr_fea=[]
for i in corr.columns:
    for j in corr.columns:
        if i!=j and abs(corr[i][j])>0.9 and ((i,j) not in corr_fea) and ((j,i) not in corr_fea):
            count+=1
            corr_fea.append((i,j))
            add_relation(corr_fea)

In [12]:
from sklearn.decomposition import PCA

new_num_fea=len(X_train.columns)-count

pca=PCA(n_components=new_num_fea)
X_train=pca.fit_transform(X_train)
X_test=pca.transform(X_test)


In [13]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
regressor=MLPRegressor(hidden_layer_sizes=(50,100,70),activation="relu",random_state=1,max_iter=1000,early_stopping=True)
regressor.fit(X_train,y_train)
y_pred=regressor.predict(X_test)
err=mean_absolute_error(y_pred,y_test)
print(err)
err=mean_absolute_error(regressor.predict(X_train),y_train)
print(err)

31.91352173398952
19.74543757427465


In [24]:
from sklearn.svm import LinearSVR
regressor=LinearSVR(max_iter=5000,C=1)
regressor.fit(X_train,y_train)
y_pred=regressor.predict(X_test)
err=mean_absolute_error(y_pred,y_test)
print(err)
err=mean_absolute_error(regressor.predict(X_train),y_train)
print(err)


ValueError: Unsupported set of arguments: The combination of penalty='l2' and loss='epsilon_insensitive' are not supported when dual=False, Parameters: penalty='l2', loss='epsilon_insensitive', dual=False